In [8]:
import math
import pandas as pd
import numpy as np
from collections import deque
from itertools import combinations
from typing import Union, List
import matplotlib.pyplot as plt

In [18]:
def toStr(vec) -> str:
    return ''.join(map(str, vec))

def countOnes(vec) -> int:
    return vec.count(1)

def toInt(vec) -> int:
    return int("".join(str(i) for i in vec), 2)

def toBinVec(x, l=0) -> list:
    result = [int(i) for i in bin(x)[2:]]
    missing_zeros = l-len(result)
    if (missing_zeros > 0):
        for i in range(missing_zeros):
            result.insert(0, 0)
    return result

def binary_permutations(lst: List[int]) -> List[List[int]]:
    result = []
    for comb in combinations(range(len(lst)), lst.count(1)):
        permutation = [0] * len(lst)
        for i in comb:
            permutation[i] = 1
        result.append(permutation)
    return result

def rotate_left_vec(vec) -> list:
    shifted = vec.copy()
    temp = deque(shifted)
    temp.rotate(-1)
    return list(temp)

def rotate_left(x, r, l) -> int:
    return (x*2**r) % (2**l-1)

def find_min_max(vec) -> Union[int,int,int,list,list]:
    L = len(vec)
    vecToInt = toInt(vec)
    min = vecToInt
    max = 0
    odd_max = 0
    vec_min = vec
    vec_max = vec
    shifted = vec.copy()
    for i in range(L):
        if vecToInt < min:
            min = vecToInt
            vec_min = shifted
        if vecToInt > max:
            max = vecToInt
            vec_max = shifted
        temp = deque(shifted)
        temp.rotate(-1)
        shifted = list(temp)
        vecToInt = toInt(shifted)
    return min, max, odd_max, vec_min, vec_max

def find_rotational_distance(vec1, vec2) -> Union[int,int]:
    L = len(vec1)
    vec1ToInt = toInt(vec1)
    vec2ToInt = toInt(vec2)
    left_shifts = 0
    shifted = vec1.copy()
    for i in range(L):
        if toInt(shifted) == vec2ToInt:
            left_shifts = i
            break
        temp = deque(shifted)
        temp.rotate(-1)
        shifted = list(temp)
    return left_shifts, L-left_shifts

def calculate_c(k: int, l: int, N1: int) -> int:
    return (k-2)*(2**l-k**N1)

def calculate_z(k: int, parity_vector: List[int]) -> int:
    indices = []
    for i, entry in enumerate(parity_vector, start=0):
        if entry == 1:
            indices.append(i)
    result = 0
    N1 = len(indices)
    for i, entry in enumerate(indices, start=0):
        result = result + k**(N1-(i+1)) * 2**(entry)
    return result

def cycle(z, c) -> list:
    result = [z]
    item = z
    while (next_item := (3*item+c)//2 if item % 2 == 1 else item // 2) not in result:
        item = next_item
        result.append(item)
    return result

def generate_df(vecs: List[List]) -> pd.DataFrame:
    df = pd.DataFrame(columns = ['vec', 'L (len)', 'N (1s)', 'min', 'max', 'odd_max', 'vec_min', 'vec_max', 'rot_left', 'quot=(N*rot+1)/L'])
    for i in range(len(vecs)):
        vec = vecs[i]
        L = len(vec)
        min, max, odd_max, vec_min, vec_max = find_min_max(vec)
        rot_dist_left, rot_dist_right = find_rotational_distance(vec_min, vec_max)
        quot = (countOnes(vec)*rot_dist_left+1)/L
        df.loc[i] = [toStr(vec), L, countOnes(vec), min, max, odd_max, toStr(vec_min), toStr(vec_max), rot_dist_left, quot]
    return df

l=14
U=8
k=3
c = calculate_c(3, l, U)
print("M({0},{1})={2}".format(l, U, M))
print("N({0},{1})={2}".format(l, U, N))
print("c={0}".format(c))

M(14,8)=21109
N(14,8)=36575
c=9823


In [25]:
#generate some cycles
rot_cases = []
max_values = []
k = 3
for i in range(1, 100):
    vec = toBinVec(i)
    x_min, x_max, odd_x_max, vec_min, vec_max = find_min_max(vec)
    rot_dist_left, rot_dist_right = find_rotational_distance(vec_min, vec_max)
    l = len(vec_max)
    N1 = countOnes(vec)
    c = calculate_c(k, l, N1)
    quot = (N1*rot_dist_left+1)/l
    if quot.is_integer() and x_max not in max_values:
        max_values.append(x_max)
        rot_cases.append([x_min, x_max, toStr(vec_min), toStr(vec_max), l, rot_dist_left, N1, quot, cycle(calculate_z(k, vec_max), c), c])

df = pd.DataFrame(rot_cases, columns = ['min', 'max', 'B_min', 'B_max', 'l', 'r', 'N1', '(N1*r+1)/l', 'cycle', 'c'])
pd.set_option('display.expand_frame_repr', False)
print(df[4:26])

    min  max    B_min    B_max  l  r  N1  (N1*r+1)/l                                  cycle    c
4     1    8     0001     1000  4  3   1         1.0                           [1, 8, 4, 2]   13
5     7   14     0111     1110  4  1   3         1.0                       [19, 23, 29, 38]  -11
6     1   16    00001    10000  5  4   1         1.0                       [1, 16, 8, 4, 2]   29
7     5   20    00101    10100  5  2   2         1.0                    [7, 22, 11, 28, 14]   23
8    11   26    01011    11010  5  3   3         2.0                   [23, 37, 58, 29, 46]    5
9    15   30    01111    11110  5  1   4         1.0                 [65, 73, 85, 103, 130]  -49
10    1   32   000001   100000  6  5   1         1.0                   [1, 32, 16, 8, 4, 2]   61
11   31   62   011111   111110  6  1   5         1.0         [211, 227, 251, 287, 341, 422] -179
12    1   64  0000001  1000000  7  6   1         1.0               [1, 64, 32, 16, 8, 4, 2]  125
13    9   72  0001001  1001000